In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import KFold, ParameterGrid

In [2]:
#Read in training and test data sets
training_data = pd.read_csv('train.csv', index_col=0)
test_data = pd.read_csv('test.csv', index_col=0)

In [3]:
training_data.head()

,Market,Day,Stock,x0,x1,x2,x3A,x3B,x3C,x3D,x3E,x4,x5,x6,y,Weight
Index,,,,,,,,,,,,,,,,
0,1,1,363,0.002696,0.530040,0.400810,1.300048e-06,4.639533e-06,1.073594e-05,2.180516e-05,4.416943e-05,0.002212,6.214737e-07,200.000000,-0.000196,1.533780
1,1,1,1223,0.081093,1.366218,1.325157,1.656861e-05,3.024668e-05,5.261872e-05,4.193762e-05,4.193762e-05,0.010932,3.414868e-05,300.000000,-0.003798,0.843604
2,1,1,1372,0.009561,0.464871,0.328008,6.500872e-08,2.323204e-07,5.182686e-07,1.080834e-06,2.030442e-06,0.000306,4.079460e-08,159.692076,-0.000003,29.824024
3,1,1,1942,0.045300,0.608222,0.456662,1.928694e-08,6.495275e-08,1.338509e-07,2.644214e-07,6.980798e-07,0.000205,9.502066e-09,212.160378,-0.000008,100.388668
4,1,1,2712,0.001605,0.476718,0.205069,6.759866e-07,2.638783e-06,6.424519e-06,1.189923e-05,2.064450e-05,0.001525,3.428403e-07,153.724351,0.000065,5.950714


In [4]:
# variables
predictor_fields = ['x0', 'x1', 'x2', 'x3A', 'x3B', 'x3D', 'x3E', 'x4', 'x5', 'x6', 'Market', 'Day', 'Stock']
best_score = 10
best_rmse_score = 10
best_params = None
best_averaged_iterations = None
early_stopping=30

In [5]:
#Cleanse any NaN, set to 0
for field in predictor_fields:
    training_data[field].fillna(0,inplace=True)
    test_data[field].fillna(0,inplace=True)

In [6]:
# Check that there are no more NaN

In [7]:
training_data.isnull().values.any()

False

In [8]:
test_data.isnull().values.any()

False

In [9]:
# Take subset of data for training & testing model
X_training_data = np.array(training_data[predictor_fields])
y_training_data = np.array(training_data['y'])
X_testing_data = np.array(test_data[predictor_fields])

In [10]:
k_fold = KFold(n_splits=10)

In [11]:
param_grid = [
              {'silent': [1],
               'eval_metric': ['rmse'],
               'eta': [0.01],
               'objective': ['reg:linear'],
               'num_round': [10000],
               'max_depth': [7],
               'min_child_weight': [5],
               'subsample': [1],
               'colsample_bytree': [1]
               }
              ]

In [12]:
for params in ParameterGrid(param_grid):
    xgboost_rounds = [] # store number of rounds taken to converge
    
    # train across k folds using number of rounds to get a prediction for given parameters 
    mse_score = []
    rmse_score = []
    
    number_of_rounds = []
    
    #Cross validate
    for train_index, test_index in k_fold.split(X_training_data):
        X_train, X_test = X_training_data[train_index], X_training_data[test_index]
        Y_train, Y_test = y_training_data[train_index], y_training_data[test_index]
    
        xg_train = xgb.DMatrix(X_train, label=Y_train)
        xg_test = xgb.DMatrix(X_test, label=Y_test)
        
        #Early stopping, uses the xg_test set to stop early if not improving
        watchlist = [(xg_train, 'train'), (xg_test, 'eval')]

        xgclassifier = xgb.train(params, xg_train, params['num_round'], evals=watchlist, early_stopping_rounds=early_stopping);

        # predict
        predicted_results = xgclassifier.predict(xg_test)
        
        # set score and optimal rounds for the fold
        mse = mean_squared_error(Y_test, predicted_results)
        mse_score.append(mse)
        rmse_score.append(np.sqrt(mse))
        number_of_rounds.append(xgclassifier.best_iteration)
        
        
    #Average score across folds
    score = np.mean(mse_score)
    root_score = np.mean(rmse_score)
    average_best_number_of_rounds = np.mean(number_of_rounds)
        
    #Update best parameters if better model found (averaged across folds) 
    if best_rmse_score > root_score: # USING RMSE AS IT'S THE LOSS FUNCTION!!! OR CHANGE THE EVALUATION FUNC TO MSE
        print(f"new best MSE Score: {score}")
        print(f"new best RMSE Score: {root_score}")
        best_score = score
        best_rmse_score = root_score
        best_params = params
        # ONLY WORK IF EARLY STOPPING ON
        print(f"Score from xgbclassifier.best_score (RMSE): {xgclassifier.best_score}")
        best_averaged_iterations = average_best_number_of_rounds
        print(f"Params: {params}")
        print(f"Number of iterations averaged over k folds: {best_averaged_iterations}")            
        print(f"Limit from xgbclassifier..best_ntree_limit: {xgclassifier.best_ntree_limit}")
        

[0]	train-rmse:0.494948	eval-rmse:0.494734
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 30 rounds.
[1]	train-rmse:0.489998	eval-rmse:0.489785
[2]	train-rmse:0.485098	eval-rmse:0.484885
[3]	train-rmse:0.480247	eval-rmse:0.480034
[4]	train-rmse:0.475445	eval-rmse:0.475232
[5]	train-rmse:0.47069	eval-rmse:0.470477
[6]	train-rmse:0.465984	eval-rmse:0.46577
[7]	train-rmse:0.461324	eval-rmse:0.461111
[8]	train-rmse:0.456711	eval-rmse:0.456497
[9]	train-rmse:0.452143	eval-rmse:0.45193
[10]	train-rmse:0.447622	eval-rmse:0.447409
[11]	train-rmse:0.443146	eval-rmse:0.442933
[12]	train-rmse:0.438714	eval-rmse:0.438501
[13]	train-rmse:0.434327	eval-rmse:0.434114
[14]	train-rmse:0.429984	eval-rmse:0.429771
[15]	train-rmse:0.425684	eval-rmse:0.425471
[16]	train-rmse:0.421427	eval-rmse:0.421214
[17]	train-rmse:0.417213	eval-rmse:0.417
[18]	train-rmse:0.413041	eval-rmse:0.412828
[19]	train-rmse:0.408911	eval-rmse:0.

[183]	train-rmse:0.078675	eval-rmse:0.078465
[184]	train-rmse:0.077889	eval-rmse:0.077679
[185]	train-rmse:0.07711	eval-rmse:0.0769
[186]	train-rmse:0.076339	eval-rmse:0.076129
[187]	train-rmse:0.075576	eval-rmse:0.075366
[188]	train-rmse:0.07482	eval-rmse:0.074611
[189]	train-rmse:0.074072	eval-rmse:0.073863
[190]	train-rmse:0.073331	eval-rmse:0.073122
[191]	train-rmse:0.072598	eval-rmse:0.072389
[192]	train-rmse:0.071872	eval-rmse:0.071664
[193]	train-rmse:0.071154	eval-rmse:0.070945
[194]	train-rmse:0.070442	eval-rmse:0.070234
[195]	train-rmse:0.069738	eval-rmse:0.06953
[196]	train-rmse:0.069041	eval-rmse:0.068833
[197]	train-rmse:0.068351	eval-rmse:0.068143
[198]	train-rmse:0.067667	eval-rmse:0.06746
[199]	train-rmse:0.066991	eval-rmse:0.066784
[200]	train-rmse:0.066321	eval-rmse:0.066114
[201]	train-rmse:0.065658	eval-rmse:0.065452
[202]	train-rmse:0.065001	eval-rmse:0.064795
[203]	train-rmse:0.064352	eval-rmse:0.064146
[204]	train-rmse:0.063708	eval-rmse:0.063503
[205]	train-rmse

[367]	train-rmse:0.012415	eval-rmse:0.012312
[368]	train-rmse:0.012291	eval-rmse:0.012189
[369]	train-rmse:0.012169	eval-rmse:0.012068
[370]	train-rmse:0.012048	eval-rmse:0.011947
[371]	train-rmse:0.011928	eval-rmse:0.011829
[372]	train-rmse:0.01181	eval-rmse:0.011711
[373]	train-rmse:0.011692	eval-rmse:0.011594
[374]	train-rmse:0.011576	eval-rmse:0.011479
[375]	train-rmse:0.011461	eval-rmse:0.011366
[376]	train-rmse:0.011347	eval-rmse:0.011253
[377]	train-rmse:0.011234	eval-rmse:0.011141
[378]	train-rmse:0.011123	eval-rmse:0.011031
[379]	train-rmse:0.011012	eval-rmse:0.010921
[380]	train-rmse:0.010903	eval-rmse:0.010812
[381]	train-rmse:0.010795	eval-rmse:0.010705
[382]	train-rmse:0.010688	eval-rmse:0.010598
[383]	train-rmse:0.010582	eval-rmse:0.010493
[384]	train-rmse:0.010477	eval-rmse:0.010389
[385]	train-rmse:0.010373	eval-rmse:0.010285
[386]	train-rmse:0.01027	eval-rmse:0.010183
[387]	train-rmse:0.010168	eval-rmse:0.010082
[388]	train-rmse:0.010067	eval-rmse:0.009981
[389]	train-

[550]	train-rmse:0.002156	eval-rmse:0.002164
[551]	train-rmse:0.002138	eval-rmse:0.002146
[552]	train-rmse:0.00212	eval-rmse:0.002129
[553]	train-rmse:0.002103	eval-rmse:0.002112
[554]	train-rmse:0.002085	eval-rmse:0.002095
[555]	train-rmse:0.002068	eval-rmse:0.002078
[556]	train-rmse:0.002051	eval-rmse:0.002062
[557]	train-rmse:0.002034	eval-rmse:0.002046
[558]	train-rmse:0.002018	eval-rmse:0.00203
[559]	train-rmse:0.002001	eval-rmse:0.002014
[560]	train-rmse:0.001985	eval-rmse:0.001998
[561]	train-rmse:0.001969	eval-rmse:0.001982
[562]	train-rmse:0.001953	eval-rmse:0.001967
[563]	train-rmse:0.001938	eval-rmse:0.001952
[564]	train-rmse:0.001922	eval-rmse:0.001937
[565]	train-rmse:0.001907	eval-rmse:0.001922
[566]	train-rmse:0.001892	eval-rmse:0.001908
[567]	train-rmse:0.001877	eval-rmse:0.001893
[568]	train-rmse:0.001862	eval-rmse:0.001879
[569]	train-rmse:0.001848	eval-rmse:0.001865
[570]	train-rmse:0.001833	eval-rmse:0.001851
[571]	train-rmse:0.001819	eval-rmse:0.001837
[572]	train-

[733]	train-rmse:0.000908	eval-rmse:0.000982
[734]	train-rmse:0.000907	eval-rmse:0.000981
[735]	train-rmse:0.000906	eval-rmse:0.00098
[736]	train-rmse:0.000905	eval-rmse:0.000979
[737]	train-rmse:0.000903	eval-rmse:0.000978
[738]	train-rmse:0.000902	eval-rmse:0.000977
[739]	train-rmse:0.000901	eval-rmse:0.000976
[740]	train-rmse:0.0009	eval-rmse:0.000976
[741]	train-rmse:0.000899	eval-rmse:0.000975
[742]	train-rmse:0.000898	eval-rmse:0.000974
[743]	train-rmse:0.000897	eval-rmse:0.000973
[744]	train-rmse:0.000896	eval-rmse:0.000972
[745]	train-rmse:0.000895	eval-rmse:0.000971
[746]	train-rmse:0.000894	eval-rmse:0.00097
[747]	train-rmse:0.000893	eval-rmse:0.00097
[748]	train-rmse:0.000892	eval-rmse:0.000969
[749]	train-rmse:0.000891	eval-rmse:0.000968
[750]	train-rmse:0.00089	eval-rmse:0.000967
[751]	train-rmse:0.00089	eval-rmse:0.000967
[752]	train-rmse:0.000889	eval-rmse:0.000966
[753]	train-rmse:0.000888	eval-rmse:0.000965
[754]	train-rmse:0.000887	eval-rmse:0.000965
[755]	train-rmse:

...

[910]	train-rmse:0.000862	eval-rmse:0.000666
[911]	train-rmse:0.000862	eval-rmse:0.000666
[912]	train-rmse:0.000862	eval-rmse:0.000666
[913]	train-rmse:0.000861	eval-rmse:0.000666
[914]	train-rmse:0.000861	eval-rmse:0.000666
[915]	train-rmse:0.000861	eval-rmse:0.000666
[916]	train-rmse:0.000861	eval-rmse:0.000666
[917]	train-rmse:0.000861	eval-rmse:0.000666
[918]	train-rmse:0.000861	eval-rmse:0.000666
[919]	train-rmse:0.000861	eval-rmse:0.000666
[920]	train-rmse:0.00086	eval-rmse:0.000666
[921]	train-rmse:0.00086	eval-rmse:0.000666
[922]	train-rmse:0.00086	eval-rmse:0.000666
[923]	train-rmse:0.00086	eval-rmse:0.000666
[924]	train-rmse:0.00086	eval-rmse:0.000666
[925]	train-rmse:0.00086	eval-rmse:0.000666
[926]	train-rmse:0.00086	eval-rmse:0.000666
[927]	train-rmse:0.000859	eval-rmse:0.000666
[928]	train-rmse:0.000859	eval-rmse:0.000666
[929]	train-rmse:0.000859	eval-rmse:0.000666
[930]	train-rmse:0.000859	eval-rmse:0.000666
[931]	train-rmse:0.000859	eval-rmse:0.000666
Stopping. Best it

[159]	train-rmse:0.10013	eval-rmse:0.100091
[160]	train-rmse:0.099129	eval-rmse:0.09909
[161]	train-rmse:0.098137	eval-rmse:0.098099
[162]	train-rmse:0.097156	eval-rmse:0.097117
[163]	train-rmse:0.096185	eval-rmse:0.096146
[164]	train-rmse:0.095223	eval-rmse:0.095184
[165]	train-rmse:0.094271	eval-rmse:0.094232
[166]	train-rmse:0.093328	eval-rmse:0.093289
[167]	train-rmse:0.092395	eval-rmse:0.092356
[168]	train-rmse:0.091471	eval-rmse:0.091432
[169]	train-rmse:0.090557	eval-rmse:0.090518
[170]	train-rmse:0.089651	eval-rmse:0.089612
[171]	train-rmse:0.088755	eval-rmse:0.088716
[172]	train-rmse:0.087867	eval-rmse:0.087828
[173]	train-rmse:0.086989	eval-rmse:0.08695
[174]	train-rmse:0.086119	eval-rmse:0.08608
[175]	train-rmse:0.085258	eval-rmse:0.085219
[176]	train-rmse:0.084405	eval-rmse:0.084366
[177]	train-rmse:0.083562	eval-rmse:0.083522
[178]	train-rmse:0.082726	eval-rmse:0.082687
[179]	train-rmse:0.081899	eval-rmse:0.081859
[180]	train-rmse:0.08108	eval-rmse:0.081041
[181]	train-rms

[343]	train-rmse:0.015784	eval-rmse:0.01573
[344]	train-rmse:0.015626	eval-rmse:0.015573
[345]	train-rmse:0.015471	eval-rmse:0.015417
[346]	train-rmse:0.015316	eval-rmse:0.015263
[347]	train-rmse:0.015164	eval-rmse:0.015111
[348]	train-rmse:0.015013	eval-rmse:0.01496
[349]	train-rmse:0.014863	eval-rmse:0.01481
[350]	train-rmse:0.014715	eval-rmse:0.014662
[351]	train-rmse:0.014569	eval-rmse:0.014515
[352]	train-rmse:0.014423	eval-rmse:0.01437
[353]	train-rmse:0.01428	eval-rmse:0.014227
[354]	train-rmse:0.014138	eval-rmse:0.014085
[355]	train-rmse:0.013997	eval-rmse:0.013944
[356]	train-rmse:0.013857	eval-rmse:0.013805
[357]	train-rmse:0.01372	eval-rmse:0.013667
[358]	train-rmse:0.013583	eval-rmse:0.01353
[359]	train-rmse:0.013448	eval-rmse:0.013395
[360]	train-rmse:0.013314	eval-rmse:0.013261
[361]	train-rmse:0.013181	eval-rmse:0.013129
[362]	train-rmse:0.01305	eval-rmse:0.012998
[363]	train-rmse:0.01292	eval-rmse:0.012868
[364]	train-rmse:0.012792	eval-rmse:0.012739
[365]	train-rmse:0.

[527]	train-rmse:0.002633	eval-rmse:0.002641
[528]	train-rmse:0.00261	eval-rmse:0.002619
[529]	train-rmse:0.002587	eval-rmse:0.002596
[530]	train-rmse:0.002564	eval-rmse:0.002574
[531]	train-rmse:0.002541	eval-rmse:0.002552
[532]	train-rmse:0.002519	eval-rmse:0.002531
[533]	train-rmse:0.002497	eval-rmse:0.00251
[534]	train-rmse:0.002475	eval-rmse:0.002489
[535]	train-rmse:0.002453	eval-rmse:0.002468
[536]	train-rmse:0.002432	eval-rmse:0.002447
[537]	train-rmse:0.00241	eval-rmse:0.002427
[538]	train-rmse:0.00239	eval-rmse:0.002407
[539]	train-rmse:0.002369	eval-rmse:0.002387
[540]	train-rmse:0.002348	eval-rmse:0.002368
[541]	train-rmse:0.002328	eval-rmse:0.002348
[542]	train-rmse:0.002308	eval-rmse:0.002329
[543]	train-rmse:0.002288	eval-rmse:0.002311
[544]	train-rmse:0.002269	eval-rmse:0.002292
[545]	train-rmse:0.00225	eval-rmse:0.002274
[546]	train-rmse:0.00223	eval-rmse:0.002256
[547]	train-rmse:0.002212	eval-rmse:0.002238
[548]	train-rmse:0.002193	eval-rmse:0.00222
[549]	train-rmse:

[710]	train-rmse:0.000941	eval-rmse:0.001136
[711]	train-rmse:0.000939	eval-rmse:0.001135
[712]	train-rmse:0.000938	eval-rmse:0.001133
[713]	train-rmse:0.000936	eval-rmse:0.001133
[714]	train-rmse:0.000934	eval-rmse:0.001131
[715]	train-rmse:0.000933	eval-rmse:0.00113
[716]	train-rmse:0.000931	eval-rmse:0.001129
[717]	train-rmse:0.000929	eval-rmse:0.001128
[718]	train-rmse:0.000928	eval-rmse:0.001127
[719]	train-rmse:0.000926	eval-rmse:0.001126
[720]	train-rmse:0.000925	eval-rmse:0.001125
[721]	train-rmse:0.000923	eval-rmse:0.001124
[722]	train-rmse:0.000922	eval-rmse:0.001123
[723]	train-rmse:0.00092	eval-rmse:0.001123
[724]	train-rmse:0.000919	eval-rmse:0.001122
[725]	train-rmse:0.000917	eval-rmse:0.001121
[726]	train-rmse:0.000916	eval-rmse:0.00112
[727]	train-rmse:0.000915	eval-rmse:0.00112
[728]	train-rmse:0.000913	eval-rmse:0.001119
[729]	train-rmse:0.000912	eval-rmse:0.001118
[730]	train-rmse:0.000911	eval-rmse:0.001118
[731]	train-rmse:0.00091	eval-rmse:0.001117
[732]	train-rms

In [13]:
#Best parameters from grid search
print("best MSE: ", best_score)
print("best RMSE: ", best_rmse_score)
print("params: ", best_params)
print("number of rounds for each fold ", number_of_rounds)
print("number of rounds (averaged across folds): ", best_averaged_iterations)

best MSE:  9.11501253765e-07
best RMSE:  0.0008637381263
params:  {'colsample_bytree': 1, 'eta': 0.01, 'eval_metric': 'rmse', 'max_depth': 7, 'min_child_weight': 5, 'num_round': 10000, 'objective': 'reg:linear', 'silent': 1, 'subsample': 1}
number of rounds for each fold  [927, 903, 858, 916, 928, 967, 930, 1010, 901, 811]
number of rounds (averaged across folds):  915.1


In [14]:
#Train XGboost regressor with optimal number of rounds and parameters on training data

In [15]:
xg_train = xgb.DMatrix(X_training_data, label=y_training_data)
xg_test = xgb.DMatrix(X_testing_data)
watchlist = [(xg_train, 'train')]

In [16]:
xgclassifier = xgb.train(best_params, xg_train, int(best_averaged_iterations), evals=watchlist)

[0]	train-rmse:0.494926
[1]	train-rmse:0.489977
[2]	train-rmse:0.485078
[3]	train-rmse:0.480227
[4]	train-rmse:0.475425
[5]	train-rmse:0.47067
[6]	train-rmse:0.465964
[7]	train-rmse:0.461304
[8]	train-rmse:0.456691
[9]	train-rmse:0.452124
[10]	train-rmse:0.447603
[11]	train-rmse:0.443127
[12]	train-rmse:0.438696
[13]	train-rmse:0.434309
[14]	train-rmse:0.429966
[15]	train-rmse:0.425666
[16]	train-rmse:0.42141
[17]	train-rmse:0.417195
[18]	train-rmse:0.413024
[19]	train-rmse:0.408893
[20]	train-rmse:0.404804
[21]	train-rmse:0.400756
[22]	train-rmse:0.396749
[23]	train-rmse:0.392781
[24]	train-rmse:0.388854
[25]	train-rmse:0.384965
[26]	train-rmse:0.381116
[27]	train-rmse:0.377304
[28]	train-rmse:0.373531
[29]	train-rmse:0.369796
[30]	train-rmse:0.366098
[31]	train-rmse:0.362437
[32]	train-rmse:0.358813
[33]	train-rmse:0.355225
[34]	train-rmse:0.351673
[35]	train-rmse:0.348156
[36]	train-rmse:0.344674
[37]	train-rmse:0.341228
[38]	train-rmse:0.337816
[39]	train-rmse:0.334437
[40]	train-r

[321]	train-rmse:0.019677
[322]	train-rmse:0.019481
[323]	train-rmse:0.019287
[324]	train-rmse:0.019094
[325]	train-rmse:0.018904
[326]	train-rmse:0.018715
[327]	train-rmse:0.018528
[328]	train-rmse:0.018344
[329]	train-rmse:0.018161
[330]	train-rmse:0.01798
[331]	train-rmse:0.0178
[332]	train-rmse:0.017623
[333]	train-rmse:0.017447
[334]	train-rmse:0.017273
[335]	train-rmse:0.017101
[336]	train-rmse:0.01693
[337]	train-rmse:0.016762
[338]	train-rmse:0.016595
[339]	train-rmse:0.016429
[340]	train-rmse:0.016265
[341]	train-rmse:0.016103
[342]	train-rmse:0.015943
[343]	train-rmse:0.015784
[344]	train-rmse:0.015627
[345]	train-rmse:0.015471
[346]	train-rmse:0.015317
[347]	train-rmse:0.015164
[348]	train-rmse:0.015013
[349]	train-rmse:0.014864
[350]	train-rmse:0.014715
[351]	train-rmse:0.014569
[352]	train-rmse:0.014424
[353]	train-rmse:0.01428
[354]	train-rmse:0.014138
[355]	train-rmse:0.013997
[356]	train-rmse:0.013858
[357]	train-rmse:0.01372
[358]	train-rmse:0.013583
[359]	train-rmse:0

[638]	train-rmse:0.001191
[639]	train-rmse:0.001186
[640]	train-rmse:0.00118
[641]	train-rmse:0.001174
[642]	train-rmse:0.001169
[643]	train-rmse:0.001164
[644]	train-rmse:0.001159
[645]	train-rmse:0.001153
[646]	train-rmse:0.001148
[647]	train-rmse:0.001143
[648]	train-rmse:0.001138
[649]	train-rmse:0.001133
[650]	train-rmse:0.001129
[651]	train-rmse:0.001124
[652]	train-rmse:0.001119
[653]	train-rmse:0.001115
[654]	train-rmse:0.00111
[655]	train-rmse:0.001106
[656]	train-rmse:0.001101
[657]	train-rmse:0.001097
[658]	train-rmse:0.001093
[659]	train-rmse:0.001089
[660]	train-rmse:0.001085
[661]	train-rmse:0.001081
[662]	train-rmse:0.001077
[663]	train-rmse:0.001073
[664]	train-rmse:0.001069
[665]	train-rmse:0.001065
[666]	train-rmse:0.001062
[667]	train-rmse:0.001058
[668]	train-rmse:0.001054
[669]	train-rmse:0.001051
[670]	train-rmse:0.001047
[671]	train-rmse:0.001044
[672]	train-rmse:0.00104
[673]	train-rmse:0.001037
[674]	train-rmse:0.001034
[675]	train-rmse:0.001031
[676]	train-rms

In [17]:
#xgclassifier = xgb.train(best_params, xg_train, best_params['num_round'], evals=watchlist, early_stopping_rounds=early_stopping)

In [18]:
predicted_y = xgclassifier.predict(xg_test) #pass in optimised parameters

In [19]:
predicted_y_df = pd.DataFrame(predicted_y)
predicted_y_df.to_csv('results/xgb_optimised_eta_0.001.csv', header=True)